In [164]:
import pickle, json

import discovery as ds

import numpy as np
import jax
import jax.numpy as jnp

import matplotlib.pyplot as pp

In [181]:
import imp

imp.reload(ds.matrix)
imp.reload(ds.signals)
imp.reload(ds.likelihood)
imp.reload(ds.prior)
imp.reload(ds.pulsar)
imp.reload(ds)

<module 'discovery' from '/Users/vallis/Documents/discovery/src/discovery/__init__.py'>

Load pulsars

In [166]:
psrfiles = !ls ../data
psrs = [ds.Pulsar.read_feather(f'../data/{psrfile}') for psrfile in psrfiles[:2]]
psrs

Standard model 3a

In [182]:
Tspan = ds.getspan(psrs)

gbl = ds.GlobalLikelihood((ds.PulsarLikelihood([psr.residuals,
                                                ds.makenoise_measurement(psr, psr.noisedict),
                                                ds.makegp_ecorr(psr, psr.noisedict),
                                                ds.makegp_timing(psr, variance=1e-14),
                                                ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='rednoise')])
                           for psr in psrs),
                          ds.makegp_fourier_global(psrs, ds.powerlaw, ds.hd_orf, 14, T=Tspan, name='gw'))

logl = gbl.logL

In [183]:
p0 = ds.sample_uniform(logl.params)
p0

{'B1855+09_rednoise_gamma': 6.337026790487537,
 'B1855+09_rednoise_log10_A': -17.100943473787986,
 'B1937+21_rednoise_gamma': 0.4449595245917949,
 'B1937+21_rednoise_log10_A': -12.859995213958111,
 'gw_log10_A': -12.968476457039607,
 'gw_gamma': 5.870155173424073}

In [184]:
logl(p0)

Array(429841.62961266, dtype=float64)

Global GP with multiple components (different spectra and different ORFs)

In [185]:
Tspan = ds.getspan(psrs)

gbl2 = ds.GlobalLikelihood((ds.PulsarLikelihood([psr.residuals,
                                                ds.makenoise_measurement(psr, psr.noisedict),
                                                ds.makegp_ecorr(psr, psr.noisedict),
                                                ds.makegp_timing(psr, variance=1e-14),
                                                ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='rednoise')])
                            for psr in psrs),
                           ds.makegp_fourier_global(psrs,
                                                    [ds.powerlaw, ds.powerlaw],
                                                    [ds.hd_orf, ds.dipole_orf],
                                                    14, T=Tspan, name='gw'))

logl2 = gbl2.logL

Note the parameter names follow from the orf functions

In [188]:
p2 = ds.sample_uniform(logl2.params)
p2

{'B1855+09_rednoise_gamma': 1.3079309930662648,
 'B1855+09_rednoise_log10_A': -11.5753220526957,
 'B1937+21_rednoise_gamma': 4.803336230638544,
 'B1937+21_rednoise_log10_A': -11.4686576928151,
 'gw_dipoleorf_gamma': 4.14579350147083,
 'gw_dipoleorf_log10_A': -16.360764555101827,
 'gw_hdorf_gamma': 0.409405763199836,
 'gw_hdorf_log10_A': -17.066065343608827}

In [189]:
logl2(p2)

Array(440901.76079345, dtype=float64)